In [1]:
import numpy as np
import sympy

In [2]:
class Wire:
    def value(self):
        raise NotImplemented

class InputWire(Wire):
    def __init__(self, val, symbol=None):
        assert val == 0 or val == 1 or val is None
        self._val = val
        self.symbol = symbol

    def value(self):
        return self._val

class OutputWire(Wire):
    def __init__(self, func, symbol=None):
        self.symbol = symbol
        self._func = func

    def value(self):
        return self._func()

class Gate:
    def __init__(self, lwire: Wire, rwire: Wire, symbol=None):
        self.symbol = symbol
        self._lwire = lwire
        self._rwire = rwire
        self._outwire = None

    def output(self) -> Wire:
        if self._outwire is not None:
            return self._outwire

        self._outwire = self._get_outwire()
        return self._outwire

    def _get_outwire(self) -> Wire:
        raise NotImplementedError

class XorGate(Gate):
    def _get_outwire(self):
        return OutputWire(
            lambda: self._lwire.value() ^ self._rwire.value(),
            symbol=self.symbol
        )

    def _linearization_coeffs(self):
        # a + b + c \in {0,2}
        return 1, 1, 1

    def _linearization_bias(self):
        return 0

class OrGate(Gate):
    def _get_outwire(self):
        return OutputWire(
            lambda: self._lwire.value() | self._rwire.value(),
            symbol=self.symbol
        )
    def _linearization_coeffs(self):
        #      !a + !b - 2!c \in {0,1}
        # <=>  1 -a  + 1 -b -2 + 2c \in {0,1}
        # <=>  -a -b +2c +0 \in {0,1}
        # <=> -2a -2b +4c \in {0,2}
        return -2, -2, 4

    def _linearization_bias(self):
        # see coeffs
        return 0

class AndGate(Gate):
    def _get_outwire(self):
        return OutputWire(
            lambda: self._lwire.value() & self._rwire.value(),
            symbol=self.symbol
        )

    def _linearization_coeffs(self):
        #     a + b - 2c \in {0,1}
        # <=> 2a + 2b - 4c \in {0,2}
        return 2, 2, -4

    def _linearization_bias(self):
        return 0

class NotXandYGate(Gate):
    """represents a gate with formula: !x and y"""
    def _get_outwire(self):
        return OutputWire(
            lambda: not self._lwire.value() and self._rwire.value(),
            symbol=self.symbol
        )

    def _linearization_coeffs(self):
        #     !a + b − 2c \in {0, 1}
        # <=> (1-a) + b - 2c \in {0,1}
        # <=> -a + b - 2c + 1 \in {0,1}
        # <=> -2a + 2b -4c + 2 \in {0, 2}
        return -2, 2, -4

    def _linearization_bias(self):
        return 2
        
class Circuit:
    def __init__(self, inputs: list[InputWire], symbol="C"):
        self._wires = inputs
        self._gates = []
        self._output_gate = None
        self.symbol = symbol

    def add_gate(self, gate: Gate):
        # TODO: add plausibility check
        assert gate._lwire in self._wires
        assert gate._rwire in self._wires
        
        self._gates.append(gate)
        self._wires.append(gate.output())

    def set_outgate(self, gate: Gate):
        assert gate in self._gates
        self._output_gate = gate

    def eval(self):
        if self._output_gate is None:
            print("no output gate set, using last one")
            return self._gates[-1].output().value()
        return self._output_gate.output().value()

    def size(self) -> (int, int):
        # m x n
        # wires x gates
        return len(self._wires), len(self._gates)

    def _gate_wire_idxs(self, gate):
        """return the indices of the gate's wires. -> left input, right input, output"""
        assert gate in self._gates
        l = self._wires.index(gate._lwire)
        r = self._wires.index(gate._rwire)
        o = self._wires.index(gate.output())

        return l, r, o

    def matrix_G(self):
        assert self._output_gate is not None, "need output gate"
        
        G = np.zeros(self.size())
        
        for i, gate in enumerate(self._gates):
            li, ri, oi = self._gate_wire_idxs(gate)
            lc, rc, oc = gate._linearization_coeffs()
            if gate == self._output_gate:
                oc -= 3
            G[li, i] = lc
            G[ri, i] = rc
            G[oi, i] = oc

        return G

    def vector_delta(self):
        assert self._output_gate is not None, "need output gate"

        _, n = self.size()
        delta = np.zeros(n)
        
        for i, gate in enumerate(self._gates):
            b = gate._linearization_bias()
            if gate == self._output_gate:
                b += 3
            delta[i] = b

        return delta

    def matrix_V(self):
        m, n = self.size()
        return np.concat([2*np.eye(m), self.matrix_G()], axis=1)

    def vector_b(self):
        m, _ = self.size()
        return np.concat([np.zeros(m), self.vector_delta()])
            
            
        

In [3]:
np.concat([np.eye(3), np.zeros((3,2))])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 2

In [4]:
for i in [0,1]:
    for j in [0, 1]:
        print(f"{i=},{j=}")
        print(f"{AndGate(InputWire(i), InputWire(j)).output().value()=}")
        print(f"{OrGate(InputWire(i), InputWire(j)).output().value()=}")
        print(f"{XorGate(InputWire(i), InputWire(j)).output().value()=}")
        print()


i=0,j=0
AndGate(InputWire(i), InputWire(j)).output().value()=0
OrGate(InputWire(i), InputWire(j)).output().value()=0
XorGate(InputWire(i), InputWire(j)).output().value()=0

i=0,j=1
AndGate(InputWire(i), InputWire(j)).output().value()=0
OrGate(InputWire(i), InputWire(j)).output().value()=1
XorGate(InputWire(i), InputWire(j)).output().value()=1

i=1,j=0
AndGate(InputWire(i), InputWire(j)).output().value()=0
OrGate(InputWire(i), InputWire(j)).output().value()=1
XorGate(InputWire(i), InputWire(j)).output().value()=1

i=1,j=1
AndGate(InputWire(i), InputWire(j)).output().value()=1
OrGate(InputWire(i), InputWire(j)).output().value()=1
XorGate(InputWire(i), InputWire(j)).output().value()=0



In [5]:
a1 = InputWire(0, symbol="a")
b1 = InputWire(1, symbol="b")

C1 = Circuit([a1,b1], symbol="C")
g = XorGate(a1,b1, symbol="c")
C1.add_gate(g)
C1.set_outgate(g)
print(C1.size())
C1._wires

(3, 1)


In [6]:
a = InputWire(0, symbol="a")
b = InputWire(1, symbol="b")
c = InputWire(1, symbol="c")
d = InputWire(1, symbol="d")

C = Circuit([a,b, c, d], symbol="C")
aorb = OrGate(a,b, symbol="a|b")
C.add_gate(aorb)
candd = AndGate(c, d, symbol="c&d")
C.add_gate(candd)

final_gate = XorGate(aorb.output(), candd.output(), symbol="o")
C.add_gate(final_gate)
C.set_outwire(final_gate.output())
C._wires

AttributeError: 'Circuit' object has no attribute 'set_outwire'

In [7]:
C1.eval()

1

In [8]:
C1.matrix_G(), C1.vector_delta()

(array([[ 1.],
        [ 1.],
        [-2.]]),
 array([3.]))

In [9]:
V1 = C1.matrix_V()
b1 = C1.vector_b()

V1, b1

(array([[ 2.,  0.,  0.,  1.],
        [ 0.,  2.,  0.,  1.],
        [ 0.,  0.,  2., -2.]]),
 array([0., 0., 0., 3.]))

In [10]:
a = np.array([0,0,1])

In [11]:
a@V1 + b1

array([0., 0., 2., 1.])

In [12]:
bla = a @ V1 + b1
bool(np.logical_or(bla == 0, bla ==2).all())

False

In [13]:
def accept(a, V, b):
    a = np.array(a)
    out = a @ V + b
    test = np.logical_or(out == 2, out == 0)
    return bool(test.all())

def test_xor():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = XorGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if l^r:
                assert accept([l, r, 1], V, b)
            else:
                assert not accept([l, r, 0], V, b)
                assert not accept([l, r, 1], V, b)

    print("xor passed")

def test_or():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = OrGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if l or r:
                assert accept([l, r, 1], V, b), [l, r, 1]
            else:
                assert not accept([l, r, 0], V, b), [l, r, 0]
                assert not accept([l, r, 1], V, b), [l, r, 1]

    print("or passed")

def test_and():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = AndGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if l and r:
                assert accept([l, r, 1], V, b), [l, r, 1]
            else:
                assert not accept([l, r, 0], V, b), [l, r, 0]
                assert not accept([l, r, 1], V, b), [l, r, 1]

    print("and passed")


def test_not_x_and_y():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = NotXandYGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if not l and r:
                assert accept([l, r, 1], V, b), [l, r, 1]
            else:
                assert not accept([l, r, 0], V, b), [l, r, 0]
                assert not accept([l, r, 1], V, b), [l, r, 1]

    print("not-x-and-y passed")

In [14]:
test_xor()
test_and()
test_or()
test_not_x_and_y()

xor passed
and passed
or passed
not-x-and-y passed


In [15]:
def lagrange_basis(nodes: list[int], var, field):
    polys = []

    for j, val_j in enumerate(nodes):
        val_j = field.convert(val_j)
        expr = field.one
        for m, val_m in enumerate(nodes):
            if m == j:
                continue
            val_m = field.convert(val_m)

            expr *= (var - val_m) * ((val_j - val_m)**-1)
        polys.append(expr.as_poly(domain=field))

    return polys

def polynomial_with_values(nodes: list[int], vals: list[int], var, field):
    assert len(nodes) == len(vals)
    
    basis = lagrange_basis(nodes, var, field)
    expr = field.zero
    for v, b in zip(vals, basis):
        expr += v * b

    return expr.as_poly()

In [16]:
x = sympy.symbols('x')
F = sympy.GF(29)

basis = lagrange_basis([0,1,2], x, field=F)
polynomial_with_values([0,1,2], [0, 1, 4], x, F)

Poly(x**2, x, modulus=29)

In [17]:
import random

def prepare_field(d: int):
    p = sympy.ntheory.generate.nextprime(max(8, d))
    field = sympy.GF(p)
    rs = []

    while len(rs) < d:
        el = field.convert(random.randrange(p))
        if el in rs:
            continue
        rs.append(el)

    return rs, field

def _ssp_polynomial_v0(rs, b: np.array, var, field):
    b_finite = [field.convert(x) for x in b-1]
    return polynomial_with_values(rs, b_finite, var, field)

def _ssp_polynomial_vi(rs, i: int, V: np.array, var, field):
    i -= 1
    vi_finite = [field.convert(x) for x in V[i]]
    return polynomial_with_values(rs, vi_finite, var, field)

In [18]:
rs, field = prepare_field(len(b))

TypeError: object of type 'InputWire' has no len()

In [19]:
v_0 = _ssp_polynomial_v0(rs, b, x, field)
v_0

NameError: name 'rs' is not defined

In [20]:
V1.shape

(3, 4)

In [21]:
for i in range(1, 4):
    v_i = _ssp_polynomial_vi(rs, i, V1, x, field)
    print(i, v_i)
    for rj in rs:
        print(v_i(rj), end=' ')
    print()

NameError: name 'rs' is not defined

In [22]:
def mega_polynomial(a: np.array, V: np.array, b: np.array):
    m, d = V.shape
    rs, field = prepare_field(d)
    x = sympy.symbols("x")
    polynomial = _ssp_polynomial_v0(rs, b, x, field)

    for i in range(m):
        a_i = a[i]
        polynomial += (a_i * _ssp_polynomial_vi(rs, i+1, V, x, field))

    return polynomial**2, rs

In [23]:
a = np.array([0,1,0])
poly, rs = mega_polynomial(a, V1, b1)
poly

Poly(4*x**6 - 3*x**5 - 4*x**4 - x**3 + 4*x + 1, x, modulus=11)

In [24]:
for a in [0,1]:
    for b in [0,1]:
        for c in [0,1]:
            print(a,b,c)
            poly, rs = mega_polynomial(np.array([a,b,c]), V1, b1)
            for rj in rs:
                print(poly(rj), end = ' ')
            print()

0 0 0
1 1 1 4 
0 0 1
1 1 1 0 
0 1 0
1 1 1 -2 
0 1 1
1 1 1 1 
1 0 0
1 1 1 -2 
1 0 1
1 1 1 1 
1 1 0
1 1 1 5 
1 1 1
1 1 1 4 


In [25]:
def target_polynomial(rs, var, field):
    expr = 1
    for rj in rs:
        expr *= (var - rj)

    return expr.as_poly(domain=field)

def divides(target, poly) -> bool:
    """true if target divides polynomial"""

    _, rem = poly.div(target)
    return rem == 0

In [26]:
t = target_polynomial(rs, x, field)
t

NameError: name 'field' is not defined

In [27]:
divides(t, poly - 1)

NameError: name 't' is not defined

In [28]:
help(poly.div)

Help on method div in module sympy.polys.polytools:

div(g, auto=True) method of sympy.polys.polytools.Poly instance
    Polynomial division with remainder of ``f`` by ``g``.

    Examples

    >>> from sympy import Poly
    >>> from sympy.abc import x

    >>> Poly(x**2 + 1, x).div(Poly(2*x - 4, x))
    (Poly(1/2*x + 1, x, domain='QQ'), Poly(5, x, domain='QQ'))

    >>> Poly(x**2 + 1, x).div(Poly(2*x - 4, x), auto=False)
    (Poly(0, x, domain='ZZ'), Poly(x**2 + 1, x, domain='ZZ'))



In [29]:
a = InputWire(None, symbol="a")
b = InputWire(None, symbol="b")

C = Circuit([a, b], symbol="C")
g = AndGate(a,b, symbol="c")
C.add_gate(g)
C.set_outgate(g)

V, b = C.matrix_V(), C.vector_b()

In [30]:
V, b

(array([[ 2.,  0.,  0.,  2.],
        [ 0.,  2.,  0.,  2.],
        [ 0.,  0.,  2., -7.]]),
 array([0., 0., 0., 3.]))

In [31]:
x1 = InputWire(None, symbol='x1')
x2 = InputWire(None, symbol='x2')
x3 = InputWire(None, symbol='x3')
x4 = InputWire(None, symbol='x4')

C = Circuit([x1, x2, x3, x4], symbol="phi")

g1 = NotXandYGate(x1, x2, symbol="g1")
C.add_gate(g1)

g2 = OrGate(x3, x4, symbol = "g2")
C.add_gate(g2)

g_out = AndGate(g1.output(), g2.output(), symbol="g_out")
C.add_gate(g_out)
C.set_outgate(g_out)

In [32]:
V, b = C.matrix_V(), C.vector_b()
V, b

(array([[ 2.,  0.,  0.,  0.,  0.,  0.,  0., -2.,  0.,  0.],
        [ 0.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.],
        [ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0., -2.,  0.],
        [ 0.,  0.,  0.,  2.,  0.,  0.,  0.,  0., -2.,  0.],
        [ 0.,  0.,  0.,  0.,  2.,  0.,  0., -4.,  0.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  4.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0., -7.]]),
 array([0., 0., 0., 0., 0., 0., 0., 2., 0., 3.]))

In [33]:
# Happy case: satisfying assignment and correct gate values
a1 = np.array([
    0, # x1
    1, # x2
    1, # x3
    1, # x4
    1, # !x1 and x2
    1, # x3 or x4
    1, # output
])

a1@V + b

array([0., 2., 2., 2., 2., 2., 2., 0., 0., 0.])

In [34]:
# Problem 1: non-satsfying assignment (but correct 'computations')
a2  = np.array([
    0, # x1
    1, # x2
    0, # x3
    0, # x4
    1, # !x1 and x2
    0, # x3 or x4
    0, # output
])
a2 @ V + b

array([0., 2., 0., 0., 2., 0., 0., 0., 0., 5.])

In [35]:
# Problem 2: satisfying assignments, but wrong computations
a3 = np.array([
    0, # x1
    1, # x2
    1, # x3
    1, # x4
    1, # !x1 and x2
    0, # x3 or x4
    0, # output
])
a3@ V + b

array([ 0.,  2.,  2.,  2.,  2.,  0.,  0.,  0., -4.,  5.])

In [36]:
# Problem 3: non-satsfying assignment and 'cheating' computations
a4 = np.array([
    0, # x1
    1, # x2
    0, # x3
    0, # x4
    1, # !x1 and x2
    0, # x3 or x4
    1, # output
])
a4 @ V + b

array([ 0.,  2.,  0.,  0.,  2.,  0.,  2.,  0.,  0., -2.])

In [37]:
x1._val = 0
x2._val = 1
x3._val = 1
x4._val = 0
C.eval()

1